# 第3章: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](https://nlp100.github.io/data/jawiki-country.json.gz)がある．

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

In [1]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz -O jawiki-country.json.gz
!gunzip jawiki-country.json.gz

--2022-11-29 19:33:21--  https://nlp100.github.io/data/jawiki-country.json.gz
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5068362 (4.8M) [application/gzip]
Saving to: ‘jawiki-country.json.gz’

jawiki-country.json 100%[===================>]   4.83M  --.-KB/s    in 0.08s   

2022-11-29 19:33:21 (57.7 MB/s) - ‘jawiki-country.json.gz’ saved [5068362/5068362]



## 20. JSONデータの読み込み
---
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [2]:
import pandas as pd

df = pd.read_json("jawiki-country.json", lines=True)
uk_df = df[df["title"] == "イギリス"]
uk_df = uk_df["text"]

print(uk_df)

235    {{redirect|UK}}\n{{redirect|英国|春秋時代の諸侯国|英 (春秋)...
Name: text, dtype: object


## 21. カテゴリ名を含む行を抽出
---
記事中でカテゴリ名を宣言している行を抽出せよ．

In [3]:
import re

pattern = r"\[\[Category:.+\]\]"
result = "\n".join(re.findall(pattern, uk_df.values[0]))
print(result)

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出
---
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [4]:
pattern = r"\[\[Category:(.+?)(?:\|.+)?\]\]"
result = "\n".join(re.findall(pattern, uk_df.values[0]))
print(result)

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


## 23. セクション構造
---
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [5]:
pattern = r"=(=+)(.+?)="
result = "\n".join([i[1] + ":" + str(len(i[0])) for i in re.findall(pattern, uk_df.values[0])])
print(result)

国名:1
歴史:1
地理:1
主要都市:2
気候:2
政治:1
元首:2
法:2
内政:2
地方行政区分:2
外交・軍事:2
経済:1
鉱業:2
農業:2
貿易:2
不動産:2
エネルギー政策:2
通貨:2
企業:2
通信:3
交通:1
道路:2
鉄道:2
海運:2
航空:2
科学技術:1
国民:1
言語:2
宗教:2
婚姻:2
移住:2
教育:2
医療:2
文化:1
食文化:2
文学:2
哲学:2
音楽:2
ポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:2
サッカー:3
クリケット:3
競馬:3
モータースポーツ:3
野球:3
 カーリング :3
 自転車競技 :3
脚注:1
関連項目:1
外部リンク:1


## 24. ファイル参照の抽出
---
記事から参照されているメディアファイルをすべて抜き出せ．

In [6]:
pattern = r"ファイル:(.+?)\|"
result = "\n".join(re.findall(pattern, uk_df.values[0]))
print(result)

Royal Coat of Arms of the United Kingdom.svg
Descriptio Prime Tabulae Europae.jpg
Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
London.bankofengland.arp.jpg
Battle of Waterloo 1815.PNG
Uk topo en.jpg
BenNevis2005.jpg
Population density UK 2011 census.png
2019 Greenwich Peninsula & Canary Wharf.jpg
Birmingham Skyline from Edgbaston Cricket Ground crop.jpg
Leeds CBD at night.jpg
Glasgow and the Clyde from the air (geograph 4665720).jpg
Palace of Westminster, London - Feb 2007.jpg
Scotland Parliament Holyrood.jpg
Donald Trump and Theresa May (33998675310) (cropped).jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow Terminal 5C Iwelumo-1.jpg
Airbus A380-841 G-XLEB British Airways (10424102995).jpg
UKpop.svg
Anglospeak.svg
Royal Aberdeen Children's Hospital.jpg
CHANDOS3.jpg
The Fabs.JPG
Wembley Stadium, illuminated.jpg


## 25. テンプレートの抽出
---
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [7]:
pattern = r"\|(.+?)\s=\s*(.+)"
template = {i[0]: i[1] for i in re.findall(pattern, uk_df.values[0])}
print(template)

{'略名 ': 'イギリス', '日本語国名': 'グレートブリテン及び北アイルランド連合王国', '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />', '国旗画像': 'Flag of the United Kingdom.svg', '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]', '国章リンク': '（[[イギリスの国章|国章]]）', '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）', '国歌': "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />''神よ女王を護り賜え''<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}", '地図画像': 'Europe-UK.svg', '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg', '公用語': '[[英語]]', '首都': '[[ロンドン]]（事実上）', '最大都市': 'ロンドン', '元首等肩書': '[[イギリスの君主|女王]]', '元首等氏名': '[[エリザベス2世]]', '首相等肩書': '[[イギリスの首相|首相]]', '首相等氏名': '[[ボリス・ジョンソン]]', '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]', '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]', '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]', '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}', '他元首等肩

## 26. 強調マークアップの除去
---
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [8]:
def strip_markup(text):
    text = re.sub(r"\'{2,}(.+?)\'{2,}", "\\1", text)
    return text

result = {k: strip_markup(v) for k, v in template.items()}
print(result)

{'略名 ': 'イギリス', '日本語国名': 'グレートブリテン及び北アイルランド連合王国', '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />', '国旗画像': 'Flag of the United Kingdom.svg', '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]', '国章リンク': '（[[イギリスの国章|国章]]）', '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）', '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}', '地図画像': 'Europe-UK.svg', '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg', '公用語': '[[英語]]', '首都': '[[ロンドン]]（事実上）', '最大都市': 'ロンドン', '元首等肩書': '[[イギリスの君主|女王]]', '元首等氏名': '[[エリザベス2世]]', '首相等肩書': '[[イギリスの首相|首相]]', '首相等氏名': '[[ボリス・ジョンソン]]', '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]', '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]', '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]', '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}', '他元首等肩書3':

## 27. 内部リンクの除去Permalink
---
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [9]:
def strip_markup(text):
    text = re.sub(r"\'{2,}(.+?)\'{2,}", "\\1", text)
    text = re.sub(r"\[\[(.+?)\]\]", "\\1", text)
    return text

result = {k: strip_markup(v) for k, v in template.items()}
print(result)

{'略名 ': 'イギリス', '日本語国名': 'グレートブリテン及び北アイルランド連合王国', '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />', '国旗画像': 'Flag of the United Kingdom.svg', '国章画像': 'ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章', '国章リンク': '（イギリスの国章|国章）', '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:Dieu et mon droit|神と我が権利）', '国歌': '女王陛下万歳|{{lang|en|God Save the Queen}}{{en icon}}<br />神よ女王を護り賜え<br />{{center|ファイル:United States Navy Band - God Save the Queen.ogg}}', '地図画像': 'Europe-UK.svg', '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg', '公用語': '英語', '首都': 'ロンドン（事実上）', '最大都市': 'ロンドン', '元首等肩書': 'イギリスの君主|女王', '元首等氏名': 'エリザベス2世', '首相等肩書': 'イギリスの首相|首相', '首相等氏名': 'ボリス・ジョンソン', '他元首等肩書1': '貴族院 (イギリス)|貴族院議長', '他元首等氏名1': ':en:Norman Fowler, Baron Fowler|ノーマン・ファウラー', '他元首等肩書2': '庶民院 (イギリス)|庶民院議長', '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}', '他元首等肩書3': '連合王国最高裁判所|最高裁判所長官', '他元首等氏名3': ':en:Brenda Hale, Baroness Hale

## 28. MediaWikiマークアップの除去
---
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [10]:
def strip_markup(text):
    text = re.sub(r"\'{2,}(.+?)\'{2,}", "\\1", text)
    text = re.sub(r"\[\[(.+?)\]\]", "\\1", text)
    text = re.sub(r"\[(.+?)\]", "\\1", text)
    text = re.sub(r"\{\{(.+?)\}\}", "\\1", text)
    return text

result = {k: strip_markup(v) for k, v in template.items()}
print(result)

{'略名 ': 'イギリス', '日本語国名': 'グレートブリテン及び北アイルランド連合王国', '公式国名': 'lang|en|United Kingdom of Great Britain and Northern Ireland<ref>英語以外での正式国名:<br />', '国旗画像': 'Flag of the United Kingdom.svg', '国章画像': 'ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章', '国章リンク': '（イギリスの国章|国章）', '標語': 'lang|fr|Dieu et mon droit<br />（フランス語:Dieu et mon droit|神と我が権利）', '国歌': '女王陛下万歳|lang|en|God Save the Queenen icon<br />神よ女王を護り賜え<br />center|ファイル:United States Navy Band - God Save the Queen.ogg', '地図画像': 'Europe-UK.svg', '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg', '公用語': '英語', '首都': 'ロンドン（事実上）', '最大都市': 'ロンドン', '元首等肩書': 'イギリスの君主|女王', '元首等氏名': 'エリザベス2世', '首相等肩書': 'イギリスの首相|首相', '首相等氏名': 'ボリス・ジョンソン', '他元首等肩書1': '貴族院 (イギリス)|貴族院議長', '他元首等氏名1': ':en:Norman Fowler, Baron Fowler|ノーマン・ファウラー', '他元首等肩書2': '庶民院 (イギリス)|庶民院議長', '他元首等氏名2': '仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle', '他元首等肩書3': '連合王国最高裁判所|最高裁判所長官', '他元首等氏名3': ':en:Brenda Hale, Baroness Hale of Richmond|ブレンダ・ヘイル', 

## 29. 国旗画像のURLを取得する
---
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して，ファイル参照をURLに変換すればよい）

In [11]:
import requests as rq

url = "https://www.mediawiki.org/w/api.php?action=query&titles=File:" + template["国旗画像"] + "&prop=imageinfo&iiprop=url&format=json"
data = rq.get(url)

pattern = f'"url":"(.+?)"'
result = re.search(pattern, data.text)
print(result.group(1))

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
